# Spotify Recommendation - Finding your music taste!

## Import necessary libraries

In [1]:
import pandas as pd
from requests_oauthlib import OAuth2Session
from oauthlib.oauth2 import BackendApplicationClient
import requests
import time
from pandas import json_normalize

## Using Credentials to access OAuth for spotify

In [2]:
# Your credentials
client_id = '0af53b02c90245839d0f6d6f3bf45fb1'
client_secret = '89783cd95eac43eebbfc7f2e2a885dbb'

# Create a session
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

# Get token for the session
token = oauth.fetch_token(
    token_url='https://accounts.spotify.com/api/token',
    client_id=client_id,
    client_secret=client_secret,
    include_client_id=True
)

## Ideas for model

Before we start, we need to consider a couple of things when recommending songs:

- Is the track available for specified country?

- Did the user dislike similar tracks or this specified track?

Features for the model:

- Maybe consider getting input from users of language/country of song you want?

## Initializing Token and Getting API Request

In [3]:
# The headers to provide the access token for authentication
headers = {
    'Authorization': f'Bearer {token["access_token"]}',     # OAuth 2.0 Bearer token for authorization
    'Content-Type': 'application/json'                      # Indicates the media type of the resource
}

In [4]:
token_expiry = token['expires_at'] # gives time of expiry in total seconds
#token_expiry_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(token_expiry)) # converting to readable time format

we need to check if time.time() is going to exceed the token_expiry_date. If so then renew the token!

In [5]:
# void function to get new token if needed
def refresh_token_if_needed():
    global token, token_expiry  # Declare token and token_expiry as global variables
    # Check if the current time is within 60 seconds of expiry or past it
    if time.time() > token_expiry - 60:
        client = BackendApplicationClient(client_id=client_id)
        oauth = OAuth2Session(client=client)
        
        # Fetch a new token and update the global token variable
        token = oauth.fetch_token(
            token_url='https://accounts.spotify.com/api/token',
            client_id=client_id,
            client_secret=client_secret,
            include_client_id=True
        )
        
        # Calculate and update the global token_expiry variable
        token_expiry = time.time() + token['expires_in']


## Recommending songs based on user chosen track

In [6]:
song_title = "Self Care"
artist_name = "Mac Miller"
# check if search is case sensitive?

function that search user defined song.

The search is not case sensitive.

Ex: artist_name = mac miller, song_title='whats the use'

It will still give the song. Original name is "Mac Miller", "What's the use?"

In [24]:
def search_song(artist_name="", song_title=""):
    url = "https://api.spotify.com/v1/search"
    refresh_token_if_needed()
    headers = {
        'Authorization': f'Bearer {token["access_token"]}',  # specify token
        'Content-Type': 'application/json'  # specify data type that you receive
    }
    # dictionary format for search parameter
    # https://developer.spotify.com/documentation/web-api/reference/search 
    # use the link above for a documentation guide for search parameters
    search_query = f'track:{song_title} artist:{artist_name}'.strip()
    search_params = {
        'q': search_query if search_query else 'year:0000',  # A default query that returns minimal results
        'type': 'track',  # specifies that you are searching for tracks
        'limit': 1  # limits the response to the top 1 most relevant result
    }

    response = requests.get(url, headers=headers, params=search_params).json()
    # You might want to add error handling here to manage potential API errors

    # Extracting the most relevant song; if no songs are found, returns None
    return response['tracks']['items'][0] if response['tracks']['items'] else None

Searching song based on song title and artist name

In [33]:
df = json_normalize(search_song('mac miller', 'whats the use'))

In [30]:
df['name']

0    What's the Use?
Name: name, dtype: object

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 29 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   artists                       1 non-null      object
 1   available_markets             1 non-null      object
 2   disc_number                   1 non-null      int64 
 3   duration_ms                   1 non-null      int64 
 4   explicit                      1 non-null      bool  
 5   href                          1 non-null      object
 6   id                            1 non-null      object
 7   is_local                      1 non-null      bool  
 8   name                          1 non-null      object
 9   popularity                    1 non-null      int64 
 10  preview_url                   1 non-null      object
 11  track_number                  1 non-null      int64 
 12  type                          1 non-null      object
 13  uri                     